In [4]:
import torch
import torch.optim as optim
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
from kvasir_capsule_dataset import get_dataloader
from trainer import train_kc_model
import gc

with_gpu = torch.cuda.is_available()

if with_gpu:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print('We are now using %s.' % device)

train_path = r"C:\Users\JadHa\Desktop\Uni\DLMI-Project\kvasir-capsule-labeled-images\dataset_train.csv"
val_path = r"C:\Users\JadHa\Desktop\Uni\DLMI-Project\kvasir-capsule-labeled-images\dataset_test.csv"
dataset_path = r"C:\Users\JadHa\Desktop\Uni\DLMI-Project\kvasir-capsule-labeled-images\labelled_images"

resnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1).to(device)

resnet.fc = nn.Linear(512, 2).to(device)

train_loader = get_dataloader(csv_path=train_path, dataset_path=dataset_path, batch_size=32, shuffle=True)
val_loader = get_dataloader(csv_path=val_path, dataset_path=dataset_path, batch_size=32, shuffle=True)

optimizer = optim.Adam(params=resnet.parameters(), lr=3e-4)

criterion = nn.CrossEntropyLoss(weight=torch.tensor([1, 100]).to(device))

We are now using cuda.


In [5]:
torch.cuda.empty_cache()
gc.collect()
train_kc_model(resnet, optimizer, criterion, train_loader, val_loader, epochs=10, device=device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 56.00 MiB. GPU 0 has a total capacty of 6.00 GiB of which 0 bytes is free. Of the allocated memory 5.27 GiB is allocated by PyTorch, and 67.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF